In [5]:
import hyperspy.api as hs

from qem.image_fitting import ImageModelFitting

%matplotlib qt

# file = '../data/STO/haadf_small.hspy'

file = '/home/zzhang/OneDrive/code/qem/data/SRO-CTO/HAADF time step .hspy'
s = hs.load(file)
image = s.data
dx = s.axes_manager[1].scale *10 # convert nm to Angstrom
image = image[500:2500,2700:3200]
image = image.max() - image
model=ImageModelFitting(image, dx=dx,units='A')

INFO:root:Elements: ['A', 'B', 'C'], the order is used for the atom types. Please initiate the correct elements in your system.


In [6]:
model.plot() # inspect the image

In [7]:
model.find_peaks(min_distance=10,threshold_rel=0.05) # find atomic columns by local maximum

INFO:root:Removing peak at (226.0, 1405.0).
INFO:root:Adding peak at (224.57426202582133, 1404.0254738965755) with atom type 0.
INFO:root:Updated peak locations.


array([[ 114.        , 1977.        ],
       [ 166.        , 1907.        ],
       [ 165.        , 1979.        ],
       ...,
       [   0.        ,  367.        ],
       [   0.        ,  439.        ],
       [ 224.57426203, 1404.0254739 ]])

In [8]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/SRO-CTO/CaTiO3.cif' 
model.map_lattice(cif_file=cif_file,elements=['Ca','Ti',"O"], reciprocal=False,region_index=0) # map the cif unitcell to the image and tile it according the local peak positions

INFO:root:Reading CIF file: /home/zzhang/OneDrive/code/qem/data/SRO-CTO/CaTiO3.cif. The elements selected are ['Ca', 'Ti', 'O'] and the elements in the CIF file are Ca4Ti4O4
/home/zzhang/OneDrive/code/qem/qem/gui_classes.py:373: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(
INFO:root:Origin selected: [ 127. 1260.]
INFO:root:Vector a selected: [101.   0.]
INFO:root:Vector b selected: [ -4. 145.]
INFO:root:In pixel: Origin: [ 127. 1260.], Vector a: [101.   0.], Vector b: [ -4. 145.]
INFO:root:In space: Origin: [ 7.06749234 70.11842793] A, Vector a: [5.62060414 0.        ] A, Vector b: [-0.22259818  8.06918417] A
INFO:root:Perfect a: [96.53356386  0.        ] pixel, Perfect b: [  0.         137.22232349] pixel by rotation of unit cell and scaling with pixel size.


AtomicColumns(lattice=Atoms(symbols='Ca279O273Ti279', pbc=False, cell=[27.824772987514734, 111.29909195005894, 5.462797], spacegroup_kinds=...), lattice_ref=Atoms(symbols='Ca279O273Ti279', pbc=False, cell=[27.824772987514734, 111.29909195005894, 5.462797], spacegroup_kinds=...), elements=['Ca', 'Ti', 'O'], tol=0, pixel_size=0.05564954597502947, reference={'origin': array([ 127., 1260.]), 'vector_a': array([96.53356386,  0.        ]), 'vector_b': array([  0.        , 137.22232349])})

In [19]:
model.view_3d() # view the 3D model using ase of Atoms object, with coordinates refined from the image

In [6]:
model.select_atoms() # select the region where has atoms

INFO:root:No atoms selected.


In [18]:
model.plot_coordinates(50) # plot the coordinates of the selected region with element labels

In [9]:
import os
!export CUDA_VISIBLE_DEVICES=0
os.environ['JAX_PLATFORMS'] = 'gpu'
model.fit_background = False
params = model.init_params(atom_size=0.5)
model.fit_random_batch(params, batch_size=1000, maxiter=100, tol=1e-2, plot=False, step_size=0.05,verbose=False,num_epoch=10) # set the maxiter and tol and num_epoch to be larger for better fitting

Fitting random batch:   0%|          | 0/1 [00:00<?, ?it/s]INFO:2025-04-25 21:24:25,625:jax._src.xla_bridge:945: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-04-25 21:24:25,629:jax._src.xla_bridge:945: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Fitting random batch: 100%|██████████| 1/1 [00:05<00:00,  5.08s/it]
INFO:root:Convergence rate for pos_x = 0.34954833984375
INFO:root:Convergence rate for pos_y = 1.8989486694335938
INFO:root:Convergence not reached


{'pos_x': array([  0.        ,   0.        ,   0.        ,   2.47454476,
          2.0204792 ,   4.87772417,   7.42961693,   1.11676431,
         11.21250629,   8.48526001,   3.71855235,  10.33733082,
         16.41270828,   7.92079496,  11.82535172,  10.74846649,
         14.38064861,   9.325881  ,   8.70716572,  16.27548981,
          5.65893316,  18.47766113,  16.65755653,  15.33400917,
         15.94148064,  13.14318848,  11.37631607,  17.49089241,
         17.08050346,  18.35697556,  11.9282341 ,  18.38925934,
         15.74116707,  19.2517128 ,  11.41495419,  24.30905342,
         24.50160217,  24.35956192,  19.888134  ,  23.60463333,
         24.16718483,  24.35983086,  23.88978958,  28.32440567,
         28.12251854,  27.01091766,  26.2759037 ,  27.3822155 ,
         26.48521423,  29.70312881,  30.56400681,  25.68764496,
         28.69324684,  32.74420166,  32.21829987,  29.69566727,
         30.85450745,  33.50870895,  31.29334831,  32.78634262,
         30.67524719,  32.01453

In [10]:
model.update_coordinates()
model.update_region_crysal_analyzer()

INFO:root:The coordinates have been updated. Current state: 1
INFO:root:Updated region 0 coordinates for crystal analyzer.


In [11]:
model.plot_fitting() # compare the fitting result with the original image

: 

In [14]:
model.plot_scs(per_element=True,s=10) # plot the scattering cross section of the elements in the selected region, show each element separately

In [14]:
model.plot_scs_histogram() # plot the histogram of the scattering cross section of the selected region

In [15]:
model.voronoi_integration() # integrate the scattering cross section of the selected region using voronoi tessellation

Calculating Voronoi:   0%|          | 0/50176 [00:00<?, ?it/s]

Integrating: 100%|██████████| 171/171 [00:00<00:00, 34911.70it/s]       


(array([1381.37641537, 1296.88321828, 1260.7864156 , 1414.13386906,
        1428.35827954, 1360.88936388, 1350.15585542, 1265.02503568,
        1372.46862922, 2896.1019463 , 2745.78269806, 2832.24957468,
        2708.00040988, 2925.79068896, 2692.26977585, 2835.91920988,
        2861.30439627, 2698.60856354, 1585.61713389, 1316.1619701 ,
        1228.01605929, 1221.59393329, 1235.99104867, 1193.25520773,
        1212.46595634, 1324.07392695, 1282.42063792, 2810.55496382,
        2706.92187683, 2744.13052672, 2582.10288107, 2729.35362981,
        2715.45199662, 2751.05225952, 2724.27803086, 2753.57969459,
        1543.70547495, 1314.49848252, 1349.40094574, 1181.1594141 ,
        1345.69640999, 1262.62800179, 1288.16125117, 1378.37708244,
        1372.76221696, 2679.99091292, 2459.67742914, 2661.45808124,
        2622.19664389, 2573.16685719, 2660.72199672, 2743.84497669,
        2854.382875  , 2642.43156528, 1291.60953221, 1079.52856163,
        1231.55984095, 1243.08019839, 1205.98523

In [16]:
model.plot_scs_voronoi(per_element=True) # plot the scattering cross section of the selected region using voronoi tessellation

In [8]:
model.region_crysal_analyzer[0].plot_strain()

/home/zzhang/miniconda3/envs/jax/lib/python3.11/site-packages/matscipy/atomic_strain.py:107: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x, residuals, rank, s = np.linalg.lstsq(dr_old[mask], dr_now[mask])


In [7]:
model.region_crysal_analyzer[0].plot_polarization(a_element='Ca', b_element='Ti',cutoff_radius=3.0,exclude_border=True,border_pixel=
50,vector_scale=30.0)

In [ ]:
polarization = model.region_crysal_analyzer[0].measure_polarization(a_element='Ca', b_element='Ti', cutoff_radius=3.0,)

In [22]:
import numpy as np
polarization['magnitude'][~np.isnan(polarization['magnitude'])].mean()*model.dx

0.08196046922617682

In [17]:
polarization.keys()

dict_keys(['positions', 'polarization', 'magnitude'])

In [7]:
model.region_crysal_analyzer[0].plot_oxygen_tilt(a_type=0, o_type=2, cutoff_radius=5.0)

In [ ]:
t['']

In [8]:
t = model.region_crysal_analyzer[0].measure_oxygen_tilt(a_type=0, o_type=2, cutoff_radius=5)

In [12]:
import numpy as np
from scipy.spatial import KDTree

def calculate_oxygen_tilt(model):
    # Extract A-site atoms (type 0)
    a_site_mask = np.array(model.atom_types) == 0
    a_site_coords = np.array(model.coordinates)[a_site_mask]
    
    if len(a_site_coords) < 2:
        raise ValueError("Need at least two A-site atoms.")
    
    # Calculate A-site direction (line or plane normal)
    if len(a_site_coords) == 2:
        # Case 1: Two A-site atoms - use their connecting line
        a_vector = a_site_coords[1] - a_site_coords[0]
        norm = np.linalg.norm(a_vector)
        if norm == 0:
            raise ValueError("A-site atoms are coincident.")
        a_direction = a_vector / norm
    else:
        # Case 2: Three+ A-site atoms - compute plane normal via PCA
        centroid = np.mean(a_site_coords, axis=0)
        centered = a_site_coords - centroid
        cov_matrix = np.cov(centered.T)
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        a_direction = eigenvectors[:, np.argmin(eigenvalues)]
        a_direction /= np.linalg.norm(a_direction)

    # Extract oxygen atoms (type 2)
    oxygen_mask = np.array(model.atom_types) == 2
    oxygen_coords = np.array(model.coordinates)[oxygen_mask]
    if len(oxygen_coords) < 2:
        raise ValueError("Need at least two oxygen atoms.")

    # Find unique nearest-neighbor oxygen pairs
    tree = KDTree(oxygen_coords)
    distances, indices = tree.query(oxygen_coords, k=2)
    pairs = {frozenset((i, j)) for i, j in indices if i != j}
    
    if not pairs:
        raise ValueError("No valid oxygen pairs found.")

    # Calculate angles between O-O vectors and A-site direction
    angles = []
    for i, j in pairs:
        o_vec = oxygen_coords[j] - oxygen_coords[i]
        o_norm = np.linalg.norm(o_vec)
        if o_norm == 0:
            continue  # Skip coincident atoms
            
        o_dir = o_vec / o_norm
        # dot_product = np.dot(a_direction, o_dir)
        dot_product = np.dot(np.array([1,0]), o_dir)
        cos_theta = np.clip(dot_product, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_theta))
        if angle > 90:
            angle = angle - 180
        angles.append(angle)  # Acute angle

    return np.array(angles)


angles = calculate_oxygen_tilt(model)

In [13]:
angles

array([ 2.81455843,  3.00490898,  3.5992859 , -3.73616831, -3.66975991,
        3.56411171, -4.68504086, -3.34454314,  4.57728064, -3.03686359,
       -4.51951689,  3.04880316,  2.4672831 ,  1.61216404,  3.53076488,
       -5.89376257,  1.73081301, -2.90986314, -3.67085617, -3.01937154,
       -5.5991893 , -4.37620338, -4.25869781, -2.91762526,  6.82200213,
       -3.02060084, -1.56804862,  1.35702516,  4.77849539, -3.89881281,
       -1.87546832,  2.10319691,  3.4500182 , -2.61803362,  2.74144264,
        4.96225421, -1.55515652, -5.38892929,  3.36425348, -5.10814266])

In [32]:
a = np.array([-0.20971,   0.54016])  
b = np.array([ 0.29029,   0.45984 ]) 
vec_a = a - b
vec_b = np.array([1,0])
angle = np.arccos(np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b)))
angle = 180- np.degrees(angle)
angle

9.126027343293316

In [ ]:
   9     O15  O  -0.20971   0.54016   0.78961 (-1  0  0)+ x, y, z
  11     O17  O   0.29029   0.45984   0.71039 ( 0  0  0)+ x, y, z


In [16]:
angles

array([ -5.81392302,   1.42200626,  -3.21322653,  -5.6638655 ,
       -37.71559313,  -2.63858962,   1.56961281,  -2.60484239,
         0.96862524,  -4.65596031,  -1.14310727, -38.56254818,
        -5.2744979 ,  -2.16542899,  -3.14390468,  -2.55330871,
         2.94769671,   2.87292634,  -1.11818486,   2.88822568,
        -1.65793464,   5.85328028,   4.46362634,  -0.74228532,
         4.56771927,   6.15196351,  -3.49325996,   3.97598408,
         5.02257692,  -2.36908667,  -2.89582234,  -4.60652577,
        -3.14253717,  -2.07440646,  -6.9199027 ,   5.49355477,
        -4.21059036,   2.40147762,  -6.28561249,  22.82350304,
        -3.39345352,  -1.57896733,  -5.23169038,  -2.72265342,
        -3.49968643,   2.17799294])